In [ ]:
#!pip install scikit-learn --force
#!pip install catboost

In [1]:
import sys
sys.path.append('../..')
import pandas as pd
import numpy as np
from pandas.tseries.offsets import MonthEnd
from datetime import datetime
import os
import importlib

pd.set_option('display.max.columns', 300)

from core.calculator.storage import ModelDB
from core.calculator.core import ForecastConfig
from core.calculator.core import TrainingManager
from core.calculator.core import ForecastConfig
from core.calculator.core import ForecastEngine

# from core.calculator.deposits import DepositsCalculationType
# from core.calculator.deposits import DepositIterativeCalculator

from core.definitions import *
# from core.project_update import load_portfolio

# from core.models import DepositModels
from warnings import filterwarnings
filterwarnings('ignore')

In [2]:
from core.models.utils import run_spark_session
spark = run_spark_session('create_ftp_t1')

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/17 13:44:34 WARN ipc.Client: Exception encountered while connecting to the server : org.apache.hadoop.ipc.RemoteException(org.apache.hadoop.ipc.StandbyException): Operation category READ is not supported in state standby. Visit https://s.apache.org/sbnn-error
23/10/17 13:44:35 WARN ipc.Client: Exception encountered while connecting to the server : org.apache.hadoop.ipc.RemoteException(org.apache.hadoop.ipc.StandbyException): Operation category READ is not supported in state standby. Visit https://s.apache.org/sbnn-error
23/10/17 13:44:47 WARN yarn.Client: Same path resource file:///opt/cloudera/parcels/AnacondaPy37/jars/spark-tree-plotting-0.2.jar added multiple times to distributed cache.
23/10/17 13:44:59 WARN ipc.Client: Exception encountered while connecting to the server : org.apache.hadoop.ipc.RemoteException(org.apache.hadoop.ipc.StandbyExceptio

## Создание таблички
Сначал заджойним и уберем пропуски у фтп ставок, затем приджойним запущенное к основной таблице

пропуски можно будетт убрать методом bfill

In [ ]:
query = """
with calendar_dates 

as 
(
select report_date from dadm_alm_sbx.t_enhanced_calendar
where report_date >= '2005-01-01'
and report_date <= '2023-09-11'
)


SELECT 
calendar_dates.report_date,
ftp.`vtb_[1095d]_ftp_rate`,
ftp.`vtb_[180d]_ftp_rate`,
ftp.`vtb_[365d]_ftp_rate`,
ftp.`vtb_[548d]_ftp_rate`,
ftp.`vtb_[730d]_ftp_rate`,
ftp.`vtb_[90d]_ftp_rate`

FROM calendar_dates

left join dadm_alm_sbx.vtb_ftp_rates ftp
on calendar_dates.report_date = ftp.report_date
ORDER BY report_date 
"""

In [ ]:
df = spark.sql(query)

In [ ]:
pd_df = df.toPandas()

In [ ]:
pd_df = pd_df.sort_values(by='report_date', ascending=False).fillna(method='bfill')

In [ ]:
sparkDF = spark.createDataFrame(pd_df)

In [ ]:
#sparkDF.write.mode('overwrite').saveAsTable('dadm_alm_sbx.vtb_ftp_rates_full')

In [ ]:
spark.catalog.refreshTable('dadm_alm_sbx.vtb_ftp_rates_full')

In [ ]:
query2 = """
--agreement_balance_amt - целевая переменная

with joined_table as
(


with filtered_table as

(
select *,
case 
when datediff(open_dt, last_renewal_dt)>10 and close_plan_dt< '2030-01-01' then date_add(close_plan_dt, -weighted_plan_period)
else open_dt
end
as open_dt_correct

from prod_dadm_alm_sbx.almde_fl_dpst_rate_bal_strt2010
where report_dt>='2023-01-01'
and report_dt<='2023-01-31'
and currency_rk='rur'
and category_full like '%dep%'
and close_fact_dt >= report_dt
and open_dt <= report_dt)


select *,
case 
when abs(1095-weighted_plan_period)<25 then `vtb_[1095d]_ftp_rate`
when abs(180-weighted_plan_period)<25 then `vtb_[180d]_ftp_rate`
when abs(365-weighted_plan_period)<25 then `vtb_[365d]_ftp_rate`
when abs(548-weighted_plan_period)<25 then `vtb_[548d]_ftp_rate`
when abs(730-weighted_plan_period)<25 then `vtb_[730d]_ftp_rate`
when abs(90-weighted_plan_period)<25 then `vtb_[90d]_ftp_rate`
end
as ftp
from filtered_table
left join dadm_alm_sbx.vtb_ftp_rates_full as ftp
on filtered_table.open_dt_correct = ftp.report_date
)


select 
report_dt, 
sum(agreement_balance_amt) as balance
--open_dt, 
--weighted_plan_period,
--open_dt_correct,
--close_plan_dt,
--ftp,
--`vtb_[90d]_ftp_rate`,
--`vtb_[180d]_ftp_rate`,
--`vtb_[365d]_ftp_rate`,
--`vtb_[548d]_ftp_rate`,
--`vtb_[730d]_ftp_rate`,
--`vtb_[1095d]_ftp_rate`



from joined_table
where ftp is not null
group by report_dt
order by report_dt

"""


In [ ]:
df2 = spark.sql(query2)

In [ ]:
pd_df2

### Извлечение запроса

In [7]:
query3 = """
with joined_table as
(


with filtered_table as

(
select *,
case 
when datediff(open_dt, last_renewal_dt)>10 and close_plan_dt< '2030-01-01' then date_add(close_plan_dt, -weighted_plan_period)
else open_dt
end
as open_dt_correct

from prod_dadm_alm_sbx.almde_fl_dpst_rate_bal_strt2010
where report_dt>='2023-01-01'
and report_dt<='2023-01-31'
and currency_rk='rur'
and category_full like '%dep%'
and close_fact_dt >= report_dt
and open_dt <= report_dt)


select *,
case 
when abs(1095-weighted_plan_period)<25 then `vtb_[1095d]_ftp_rate`
when abs(180-weighted_plan_period)<25 then `vtb_[180d]_ftp_rate`
when abs(365-weighted_plan_period)<25 then `vtb_[365d]_ftp_rate`
when abs(548-weighted_plan_period)<25 then `vtb_[548d]_ftp_rate`
when abs(730-weighted_plan_period)<25 then `vtb_[730d]_ftp_rate`
when abs(90-weighted_plan_period)<25 then `vtb_[90d]_ftp_rate`
end
as ftp,

case 
when abs(1095-weighted_plan_period)<25 then 1095
when abs(180-weighted_plan_period)<25 then 180
when abs(365-weighted_plan_period)<25 then 365
when abs(548-weighted_plan_period)<25 then 548
when abs(730-weighted_plan_period)<25 then 730
when abs(90-weighted_plan_period)<25 then 90
end
as maturity,

case 
when category_full='dep_repl' or category_full='dep_repl_sub' then 1
else 0 
end 
as replenishable_flg,

case 
when category_full='dep_sub' or category_full='dep_repl_sub' then 1
else 0 
end 
as subtraction_flg



from filtered_table
left join dadm_alm_sbx.vtb_ftp_rates_full as ftp
on filtered_table.open_dt_correct = ftp.report_date


)


select 
report_dt,
agreement_balance_amt,
case
when ftp is Null then 0
else ftp
end
as ftp,
spread as spread


from
(
select joined_table.*,
case 
when maturity=90 and `date` is not NULL then 90_spread*100
when maturity=180 and `date` is not NULL then 180_spread*100
when maturity=365 and `date` is not NULL then 365_spread*100
when maturity=548 and `date` is not NULL then 548_spread*100
when maturity=730 and `date` is not NULL then 730_spread*100
when maturity=1095 and `date` is not NULL then 1095_spread*100
else 0
end 
as spread

from joined_table

left join dadm_alm_sbx.vtb_spreads_deposits as spreads
on joined_table.open_dt_correct = spreads.`date`
and joined_table.replenishable_flg = spreads.replenishable_flg
and joined_table.subtraction_flg = spreads.subtraction_flg

) tmp

"""

In [8]:
sp_df = spark.sql(query3)

In [9]:
sp_df.write.mode('overwrite').saveAsTable('dadm_alm_sbx.t_tmp_res_tnd_dep_v3')

[Stage 0:>                                                        (0 + 28) / 45]23/10/17 13:48:14 ERROR cluster.YarnScheduler: Lost executor 2 on p0drp0-db5033xp.region.vtb.ru: Unable to create executor due to Address already in use: Service 'org.apache.spark.network.netty.NettyBlockTransferService' failed after 5 retries (starting from 63535)! Consider explicitly setting the appropriate port for the service 'org.apache.spark.network.netty.NettyBlockTransferService' (for example spark.ui.port for SparkUI) to an available port or increasing spark.port.maxRetries.


In [3]:
spark

In [10]:
df2 = spark.table('dadm_alm_sbx.t_tmp_res_tnd_dep_v3')

In [11]:
df2.printSchema()

root
 |-- report_dt: timestamp (nullable = true)
 |-- agreement_balance_amt: decimal(23,5) (nullable = true)
 |-- ftp: double (nullable = true)
 |-- spread: double (nullable = true)



In [12]:
from pyspark.sql.types import DecimalType

In [13]:
# меняем схему

df2 = df2.withColumn('ftp1', df2['ftp'].cast(DecimalType(23,5)))
df2 = df2.withColumn('spread1', df2['spread'].cast(DecimalType(23,5)))

In [14]:
df2.printSchema()

root
 |-- report_dt: timestamp (nullable = true)
 |-- agreement_balance_amt: decimal(23,5) (nullable = true)
 |-- ftp: double (nullable = true)
 |-- spread: double (nullable = true)
 |-- ftp1: decimal(23,5) (nullable = true)
 |-- spread1: decimal(23,5) (nullable = true)



In [15]:
df2.show()

[Stage 0:>                                                          (0 + 1) / 1]

+-------------------+---------------------+-----+-------------------+--------+-------+
|          report_dt|agreement_balance_amt|  ftp|             spread|    ftp1|spread1|
+-------------------+---------------------+-----+-------------------+--------+-------+
|2023-01-16 00:00:00|         967548.47000|  8.0|0.13999999999999999| 8.00000|0.14000|
|2023-01-14 00:00:00|           1199.39000|  7.6|               0.61| 7.60000|0.61000|
|2023-01-07 00:00:00|         235000.00000|  8.0|                0.2| 8.00000|0.20000|
|2023-01-28 00:00:00|         205275.01000|  8.0|                0.0| 8.00000|0.00000|
|2023-01-18 00:00:00|         528567.12000| 8.25|                0.0| 8.25000|0.00000|
|2023-01-25 00:00:00|        1072868.50000|  8.0|                0.0| 8.00000|0.00000|
|2023-01-18 00:00:00|         208313.23000|  8.0|                0.0| 8.00000|0.00000|
|2023-01-09 00:00:00|        1500000.00000| 8.25|                0.0| 8.25000|0.00000|
|2023-01-17 00:00:00|          54628.09000|

In [15]:
df2 = df2.withColumn('delta1', df2['ftp1'] - df2['spread1'])

In [16]:
df2 = df2.withColumn('balance_x_ftp', df2['delta1'] * df2['agreement_balance_amt'])

In [17]:
import pyspark.sql.functions as f

In [18]:
tmp = df2.groupBy('report_dt').agg((f.sum('balance_x_ftp')/f.sum('agreement_balance_amt')).alias('weighted_rate'), f.sum('agreement_balance_amt').alias('balance'))

In [20]:
tmp.show()

+-------------------+-------------+-------------------+
|          report_dt|weighted_rate|            balance|
+-------------------+-------------+-------------------+
|2023-01-05 00:00:00|     8.993548|2854769817711.27000|
|2023-01-20 00:00:00|     8.951083|2945331175795.57000|
|2023-01-08 00:00:00|     8.993178|2859336559709.23000|
|2023-01-17 00:00:00|     8.960339|2924235969047.83000|
|2023-01-26 00:00:00|     8.936219|2944653859925.65000|
|2023-01-06 00:00:00|     8.993180|2856608049264.13000|
|2023-01-23 00:00:00|     8.948751|2944642834031.13000|
|2023-01-01 00:00:00|     8.995965|2841777066195.82000|
|2023-01-13 00:00:00|     8.983424|2874665680674.62000|
|2023-01-22 00:00:00|     8.950401|2948370173488.47000|
|2023-01-21 00:00:00|     8.950560|2947572208504.23000|
|2023-01-27 00:00:00|     8.931322|2948099584055.09000|
|2023-01-02 00:00:00|     8.994704|2846112716320.90000|
|2023-01-14 00:00:00|     8.982986|2877194558750.68000|
|2023-01-10 00:00:00|     8.986914|2859623938651

In [21]:
tmp_df = tmp.toPandas()

In [22]:
tmp_df.sort_values(by='report_dt')

,report_dt,weighted_rate,balance
7,2023-01-01,8.995965,2841777066195.82000
12,2023-01-02,8.994704,2846112716320.90000
24,2023-01-03,8.993645,2848976060265.15000
28,2023-01-04,8.993524,2852350030171.56000
0,2023-01-05,8.993548,2854769817711.27000
5,2023-01-06,8.993180,2856608049264.13000
15,2023-01-07,8.993059,2857894139313.47000
2,2023-01-08,8.993178,2859336559709.23000
18,2023-01-09,8.987748,2851436383165.84000
14,2023-01-10,8.986914,2859623938651.42000


In [23]:
tmp_df['weighted_rate'].mean()*0.95

8.518824039999998

In [24]:
tmp_df.to_csv('res_jun_ftp_v2.csv')